This notebook is for the Math 302 data project.
Created by Alice Lundgre, Gideon Hale, and Jared Winn

In [1]:
import pandas as pd
import numpy as np

The first section of this notebook deals with loading and cleaning the data.
We have chosen to us the US traffic accidents dataset, and downloaded it from Kaggle. 
To apply this to another dataset you must download a CSV version of your dataset and change the file paths. However, be aware that much of the cleaning and reformating we do here is not universal, we treat different features of the dataset differently, and often specify rule for cleaning by feature. 

In [2]:
# Replace with the path to desired dataset.
path = "US_Accidents_March23.csv"
# Note that the datset is very large, as such execution of this cell can take some time. 
df = pd.read_csv(path)

Now that we have loaded the data we wish to study and clean it. 

In [3]:
print(df.shape)
df.head()

(7728394, 46)


,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


We find that the data has 7728394 rows and 46 columns. We can already see that some of the columns will need to be trimmed (ie ID and source) and that there are several NaN values which we will need to take care of.  

In [4]:
# We drop the ID and source columns
df = df.drop(columns=["ID", "Source"])

Looking at the attributes summary on Kaggle we identify more columns to remove:
* Description ~ Natural language not parsable by simple ML techniques
* Country ~ All data is from the United States
* Timezone ~ Redundent given we already have a precise lat-long position
* Airport_Code ~ Redundent given we already have a precise lat-long position
* Street ~ Does not provide information that can be easily used by a simple ML model



We will keep several other columns, such as City, State, ect to determine how they correlate to accidents. 

In [5]:
df = df.drop(columns=["Description", "Country", "Timezone", "Airport_Code","Street" ])

Now we want to get an idea of how much data is missing, specifically if there are any rows or columns that are almost completely empty. 

In [6]:
rows, cols = df.shape
missing = df.isna()
counts = missing.sum()
for col, cnt in counts[counts > 0].sort_values().items():
    pct = (cnt / rows)
    print(f"{col}: missing {pct:.3f} %")

City: missing 0.000 %
Zipcode: missing 0.000 %
Astronomical_Twilight: missing 0.003 %
Civil_Twilight: missing 0.003 %
Sunrise_Sunset: missing 0.003 %
Nautical_Twilight: missing 0.003 %
Weather_Timestamp: missing 0.016 %
Pressure(in): missing 0.018 %
Temperature(F): missing 0.021 %
Weather_Condition: missing 0.022 %
Humidity(%): missing 0.023 %
Wind_Direction: missing 0.023 %
Visibility(mi): missing 0.023 %
Wind_Speed(mph): missing 0.074 %
Wind_Chill(F): missing 0.259 %
Precipitation(in): missing 0.285 %
End_Lng: missing 0.440 %
End_Lat: missing 0.440 %


In [7]:
# What would happen if we drop every row with Nan values?
test = df.dropna()
print(test.shape)

(3562877, 39)


We identify the following columns as potential problems:
* Wind Chill
* Precipitation
* End_Lng
* End_lat

Even though we could drop every entry with NaN values and still have a sizable dataset, for the sake of this assingment we will find other ways to clean the data

In [8]:
# Inspect values of Wind_Chill
print(df['Wind_Chill(F)'].describe())

# Let's see how many entries match Temperature
print((df['Wind_Chill(F)'] == df['Temperature(F)']).sum())

# It seems reasonable to replace NaN Wind_Chill values with the corrosponding value in Temperature. 
df['Wind_Chill(F)'] = df['Wind_Chill(F)'].fillna(df['Temperature(F)'])

count    5.729375e+06
mean     5.825105e+01
std      2.238983e+01
min     -8.900000e+01
25%      4.300000e+01
50%      6.200000e+01
75%      7.500000e+01
max      2.070000e+02
Name: Wind_Chill(F), dtype: float64
4429402


In [9]:
# Inspect values of Precipitation 
print(df['Precipitation(in)'].describe())

# Let's see how many entries are zero
print((df['Precipitation(in)'] == 0).sum())

# It seems pretty safe to assume that NaN values of preciptation are 0
df['Precipitation(in)'] = df['Precipitation(in)'].fillna(0) 

count    5.524808e+06
mean     8.407210e-03
std      1.102246e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.647000e+01
Name: Precipitation(in), dtype: float64
4991718


We now only have the ending latitude and longitude to deal with. Since the distance of the effect on traffic is already recorded, and we have the initial point, and a large portion of the dataset has Nan Values in these columns, it makes sense to delete them. 

In [10]:
df = df.drop(columns = ["End_Lng", "End_Lat"])

Now we can clean up the rest of the NaNs without loosing too much of the dataset. 

In [11]:
df = df.dropna()

In [12]:
# Number of entries saved through our data cleaning
print(df.shape[0] - test.shape[0])

3498896


Next we move on to checking if the data is in the correct format, if there are duplicates, and if all the values fall in the correct ranges. 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7061773 entries, 2 to 7728393
Data columns (total 37 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Severity               int64  
 1   Start_Time             object 
 2   End_Time               object 
 3   Start_Lat              float64
 4   Start_Lng              float64
 5   Distance(mi)           float64
 6   City                   object 
 7   County                 object 
 8   State                  object 
 9   Zipcode                object 
 10  Weather_Timestamp      object 
 11  Temperature(F)         float64
 12  Wind_Chill(F)          float64
 13  Humidity(%)            float64
 14  Pressure(in)           float64
 15  Visibility(mi)         float64
 16  Wind_Direction         object 
 17  Wind_Speed(mph)        float64
 18  Precipitation(in)      float64
 19  Weather_Condition      object 
 20  Amenity                bool   
 21  Bump                   bool   
 22  Crossing               

We see that the time columns are objects, but we need to check that they are date time objects

In [14]:
df['Start_Time'].head()

2    2016-02-08 06:49:27
3    2016-02-08 07:23:34
4    2016-02-08 07:39:07
5    2016-02-08 07:44:26
6    2016-02-08 07:59:35
Name: Start_Time, dtype: object

In [15]:
df['End_Time'].head()

2    2016-02-08 07:19:27
3    2016-02-08 07:53:34
4    2016-02-08 08:09:07
5    2016-02-08 08:14:26
6    2016-02-08 08:29:35
Name: End_Time, dtype: object

In [16]:
df['Weather_Timestamp'].head()

2    2016-02-08 06:56:00
3    2016-02-08 07:38:00
4    2016-02-08 07:53:00
5    2016-02-08 07:51:00
6    2016-02-08 07:56:00
Name: Weather_Timestamp, dtype: object

So we need to convert to a date time object 

In [17]:
# Convert to date time object
df['Start_Time'] = df['Start_Time'].str.replace(r'\.\d+', '', regex=True)
df['End_Time']   = df['End_Time'].str.replace(r'\.\d+', '', regex=True)
df['Weather_Timestamp']   = df['Weather_Timestamp'].str.replace(r'\.\d+', '', regex=True)

df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time']   = pd.to_datetime(df['End_Time'])
df['Weather_Timestamp'] = pd.to_datetime(df['Weather_Timestamp'])

Next we will check if there are duplicate entries

In [18]:
print(df.duplicated().sum())

132349


This is quite a lot, we checked, and before any cleaning was done there were 0 duplicate entries, after removing the ID and source there were 102338, and after removing the description column there were 121499. 

Given these numbers, it seems likely that the three seperate sources often recorded the same accident, with differences in the source and description column accounting for almost all of the duplicates. As such we will drop them. 

In [19]:
df = df.drop_duplicates()

Next we will check that the entries in each column make sense.
Some googling reveals the following facts:
* The min and max recorded temps in the continental US are -70 F, 134 F
* The min windchill recorded in US is -108 F
* The highest windspeed between 2016 and 2023 was 165 mph

While it is possible that values under these limits are also errors, we can remove anything outside of these ranges without any trepidation. 

In [20]:
df = df[(df['Temperature(F)'] >= -70) & (df['Temperature(F)'] <= 134)]
df = df[(df['Wind_Chill(F)'] >= -108) & (df['Wind_Chill(F)'] <= 134)]
df = df[(df['Wind_Speed(mph)'] <= 165)]

As we perform more analysis we my further clean the data, or for example create new combinations such as the amount of time that traffic was effected, or the time between the weather report and the accident, but for now this cleaning gives us what we need to start visualizing our data. 

Since these steps take a considerable amount of time to run, we will save a cleaned version of the dataset that can be loaded without running the above cells. 

In [21]:
df.to_csv('Cleaned.csv', index=False)

In [22]:
# Now we load the cleaned data checkpoint
path = "Cleaned.csv"
# Note that the datset is very large, as such execution of this cell can take some time. 
# But its much faster than the full version
data = pd.read_csv(path)

In [23]:
print(data.shape)

(6929344, 37)


**Visualizations**

Now that our data is clean, we are ready to create some visualizations in order to understand its structure. 

Geographic Distribution

In [24]:
# Figure similar to the covid heat map we made in the lab

Temporal distribution 

In [25]:
# Animation of geographic distribution, but as time move forwards each accident that happened that day is marked. 

K-Means Clustering

In [26]:
# Figures for several values of k

PCA 

In [27]:
# PCA with various numbers of components 

Column statistics

In [28]:
# Figures for several columns showing histograms of values 

**Statistics**

We have qualitatively analyzed our data with figures, now we will explore quantitative attributes. 

In [29]:
# Cells for analysis of mean, mode, range ect, ect, ect

**Model and Prediction**

In [30]:
# Cells for building models and making predictions 